# Static Lineout Notebook

## Import python libraries

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import xrayscatteringtools as xrst
xrst.enable_underscore_cleanup()

## Next define keys (data parameters) to be loaded and define function to load data
- keys_to_combine: some keys loaded for each shot & stored per shot
- keys_to_sum: some keys loaded per each run and added
- keys_to_check : check if some keys exist and have same values in all runs and load these keys 

# Load the data into defined variables
Sometimes keys of a data parameter may be saved with different name, print out all keys to figure out the required key

In [ ]:
###############################################
runNumber = 1  # Enter a single run number here
folders = xrst.get_data_paths(runNumber) # Defaults to the info in config.yaml. You can overwrite this with strings, character arrays, or lists of either.
runtype = xrst.get_config_for_runs(runNumber,'samples','sample') # Edit this for the type of run, or auto load from config.yaml
###############################################
# (1) keys_to_combine: some keys loaded for each shot & stored per shot 
# (2) keys_to_sum: some keys loaded per each run and added 
# (3) keys_to_check : check if some keys exist and have same values in all runs and load these keys 
keys_to_combine = [
    'jungfrau4M/azav_azav',
    'lightStatus/laser',
    'lightStatus/xray'
]

keys_to_sum = [
    'Sums/jungfrau4M_calib', # No lower ADU threshold
    'Sums/jungfrau4M_calib_xrayOn_thresADU1' # X-rays on, ADU lower threshold
]

keys_to_check = [
    'UserDataCfg/jungfrau4M/azav__azav_q',
    'UserDataCfg/jungfrau4M/cmask',
]
# Load the data in
_data = xrst.combineRuns(runNumber, folders, keys_to_combine, keys_to_sum, keys_to_check, verbose=False)  # this is the function to load the data with defined keys

azav = np.squeeze(_data['jungfrau4M/azav_azav']) # I(q) : 1D azimuthal average of signals in each q bin
q = _data['UserDataCfg/jungfrau4M/azav__azav_q'] # q bins 
laserOn = _data['lightStatus/laser'].astype(bool)  # laser on events 
xrayOn = _data['lightStatus/xray'].astype(bool)  # xray on events
jungfrau_sum = _data['Sums/jungfrau4M_calib_xrayOn_thresADU1']  # Total Jungfrau detector counts summed in a run
cmask = _data['UserDataCfg/jungfrau4M/cmask'] # Combined mask

# Plot the Masked Jungfrau4M Sum as a check, if you want

In [ ]:
###############################################
if True: # For plotting or not
###############################################
    _masked_sum = np.copy(jungfrau_sum)
    _masked_sum[~cmask.astype(bool)] = np.nan
    plt.figure(figsize=(10,8))
    pcm = xrst.plot_j4m(_masked_sum,vmin=0,vmax=np.nanpercentile(_masked_sum,99.5))
    plt.colorbar(pcm)
    plt.title(f'{xrst.get_config("expNumber")} Run {runNumber}: {runtype}')
    plt.show()

# Plot Single Run average azav

In [ ]:
azav_avg = np.nanmean(azav[xrayOn],axis=0) # Averaging the shot to shot azav data, no other normalization done here. This is purely to measure average signal per shot.
plt.plot(q, np.log10(azav_avg)) # Plotting it
plt.xlabel('q')
plt.ylabel('log[I]')
plt.title(f'{xrst.get_config("expNumber")} Run {runNumber}: {runtype}')
plt.show()

## Save the azav data to a folder

In [ ]:
##########################
runFolder = 'Azavs' # Edit this if you want
##########################
os.makedirs(runFolder, exist_ok=True)
np.savez(f'{runFolder}/Run_{runNumber}_{runtype}.npz',x=q,y=np.log10(azav_avg))

## Plot data for each run in the folder

In [ ]:
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import xrayscatteringtools as xrst

runFolder = 'Azavs'  # Edit this if you want
plt.figure()

# Regular expression pattern for matching filenames
pattern = re.compile(r'^Run_(\d+)_(.+?)\.npz$')

# --- BUILD A SORTABLE LIST OF (run_number, filename) ---
files = []
for filename in os.listdir(runFolder):
    match = pattern.match(filename)
    if match:
        run_number = int(match.group(1))
        files.append((run_number, filename))
    else:
        print(f"Skipped File: {filename}")

# --- SORT BY RUN NUMBER ---
files.sort(key=lambda x: x[0])

# Loop through sorted files
for run_number, filename in files:
    title = filename.replace(f"Run_{run_number}_", "").replace(".npz", "")
    file_path = os.path.join(runFolder, filename)
    data = np.load(file_path)

    plt.plot(data['x'], data['y'],
             linestyle='-', marker='o', linewidth=1.5,
             label=f'Run {run_number} {title}')

# Make plot look nice
plt.xlabel(r'q $\AA^{-1}$')
plt.ylabel('log[I]')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title(f'{xrst.get_config("expNumber")} Runs')
plt.show()